In [1]:
import sys
sys.path.extend(['infinite-zoom-stable-diffusion/'])
from helpers import *
from diffusers import StableDiffusionInpaintPipeline, EulerAncestralDiscreteScheduler
from PIL import Image
import gradio as gr
import numpy as np
import os
import time

import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

/opt/conda/lib/python3.10/site-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


#### Major Dependencies

###### May not need scipy and other libraries

In [2]:
# %pip install gradio > /dev/null 2>&1
# %pip install -qq transformers scipy ftfy accelerate > /dev/null 2>&1
# %pip install -qq --upgrade diffusers[torch] > /dev/null 2>&1
# %pip install torchaudio==2.2.0 --index-url https://download.pytorch.org/whl/cu118 > /dev/null 2>&1
# %pip install tortoise-tts==3.0.0 > /dev/null 2>&1
# %pip install deepspeed > /dev/null 2>&1

In [3]:
# This will download all the models used by Tortoise from the HF hub.
# tts = TextToSpeech()
# If you want to use deepspeed the pass use_deepspeed=True nearly 2x faster than normal
tts = TextToSpeech(use_deepspeed=True, kv_cache=True)

Some weights of the model checkpoint at jbetker/wav2vec2-large-robust-ft-libritts-voxpopuli were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at jbetker/wav2vec2-large-robust-ft-libritts-voxpopuli and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.o

[2024-03-09 09:17:56,824] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-03-09 09:17:57,035] [INFO] [logging.py:96:log_dist] [Rank -1] DeepSpeed info: version=0.14.0, git-hash=unknown, git-branch=unknown
[2024-03-09 09:17:57,036] [WARNING] [config_utils.py:69:_process_deprecated_field] Config parameter mp_size is deprecated use tensor_parallel.tp_size instead
[2024-03-09 09:17:57,037] [INFO] [logging.py:96:log_dist] [Rank -1] quantize_bits = 8 mlp_extra_grouping = False, quantize_groups = 1


Using /home/jupyter/.cache/torch_extensions/py310_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/jupyter/.cache/torch_extensions/py310_cu118/transformer_inference/build.ninja...
Building extension module transformer_inference...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.
Time to load transformer_inference op: 0.08418154716491699 seconds
[2024-03-09 09:17:57,641] [INFO] [logging.py:96:log_dist] [Rank -1] DeepSpeed-Inference config: {'layer_id': 0, 'hidden_size': 1024, 'intermediate_size': 4096, 'heads': 16, 'num_hidden_layers': -1, 'dtype': torch.float32, 'pre_layer_norm': True, 'norm_type': <NormType.LayerNorm: 1>, 'local_rank': -1, 'stochastic_mode': False, 'epsilon': 1e-05, 'mp_size': 1, 'scale_attention': True, 'triangular_masking': True, 'local_attention': False, 'window_size': 1, 'rotary_dim': -1, 'rotate_half': False, 'rotate_every_two': True, 'return_tuple': True, 'mlp_after_attn': True, 'mlp_act_func_type': <ActivationFuncType.GELU: 1>, 'specialized_mode': False, 'training_mp_size': 1, 'bigscience_bloom': False, 'max_out_tokens': 1024, 'min_out_tokens': 1, 'scale_attn_by_inverse_layer_idx': False, 'enable_qkv_quantization': False, 'use_mup': False, 'return_single_tuple': False, 'set_empty_params': False, 'transposed_mode':

Loading extension module transformer_inference...
/opt/conda/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [4]:
def zoom(
    prompts_array,
    negative_prompt,
    num_outpainting_steps,
    guidance_scale,
    num_inference_steps,
    custom_init_image
):
    prompts = {}
    for idx in range(len(prompts_array)):
        try:
            key = idx
            value = prompts_array[idx]
            prompts[key] = value
        except ValueError:
            pass


    height = 512
    width = height

    current_image = Image.new(mode="RGBA", size=(height, width))
    mask_image = np.array(current_image)[:, :, 3]
    mask_image = Image.fromarray(255-mask_image).convert("RGB")
    current_image = current_image.convert("RGB")
    if (custom_init_image):
        current_image = custom_init_image.resize(
            (width, height), resample=Image.LANCZOS)
    else:
        init_images = pipe(prompt=prompts[0],
                           negative_prompt=negative_prompt,
                           image=current_image,
                           guidance_scale=guidance_scale,
                           height=height,
                           width=width,
                           mask_image=mask_image,
                           num_inference_steps=num_inference_steps)[0]
        current_image = init_images[0]
    mask_width = 128
    num_interpol_frames = 30

    all_frames = []
    all_frames.append(current_image)

    for i in range(num_outpainting_steps):
        print('Outpaint step: ' + str(i+1) +
              ' / ' + str(num_outpainting_steps))

        prev_image_fix = current_image

        prev_image = shrink_and_paste_on_blank(current_image, mask_width)

        current_image = prev_image

        # create mask (black image with white mask_width width edges)
        mask_image = np.array(current_image)[:, :, 3]
        mask_image = Image.fromarray(255-mask_image).convert("RGB")

        # inpainting step
        current_image = current_image.convert("RGB")
        images = pipe(prompt=prompts[max(k for k in prompts.keys() if k <= i)],
                      negative_prompt=negative_prompt,
                      image=current_image,
                      guidance_scale=guidance_scale,
                      height=height,
                      width=width,
                      # generator = g_cuda.manual_seed(seed),
                      mask_image=mask_image,
                      num_inference_steps=num_inference_steps)[0]
        current_image = images[0]
        current_image.paste(prev_image, mask=prev_image)

        # interpolation steps bewteen 2 inpainted images (=sequential zoom and crop)
        for j in range(num_interpol_frames - 1):
            interpol_image = current_image
            interpol_width = round(
                (1 - (1-2*mask_width/height)**(1-(j+1)/num_interpol_frames))*height/2
            )
            interpol_image = interpol_image.crop((interpol_width,
                                                  interpol_width,
                                                  width - interpol_width,
                                                  height - interpol_width))

            interpol_image = interpol_image.resize((height, width))

            # paste the higher resolution previous image in the middle to avoid drop in quality caused by zooming
            interpol_width2 = round(
                (1 - (height-2*mask_width) / (height-2*interpol_width)) / 2*height
            )
            prev_image_fix_crop = shrink_and_paste_on_blank(
                prev_image_fix, interpol_width2)
            interpol_image.paste(prev_image_fix_crop, mask=prev_image_fix_crop)

            all_frames.append(interpol_image)
        all_frames.append(current_image)
        # interpol_image.show()
    video_file_name = "test_dummy"
    fps = 30
    save_path = video_file_name + ".mp4"
    start_frame_dupe_amount = 15
    last_frame_dupe_amount = 15

    write_video(save_path, all_frames, fps, False,
                start_frame_dupe_amount, last_frame_dupe_amount)
    return save_path

In [5]:
import openai

API_TYPE = 'azure'
API_BASE = 'https://roposo01.openai.azure.com/'
API_VERSION = '2023-03-15-preview'
AZURE_OPENAI_KEY = 'f0af22431e2b4ba880996fedb93e6d05'
ENGINE = 'gpt35-turbo'
DEFAULT_RESPONSE = 'nothing'
TEMPERATURE = 0
MAX_TOKENS = 1024
TOP_P = 1.0
FREQUENCY_PENALTY = 0
PRESENCE_PENALTY = 0
STOP = None

openai.api_type = API_TYPE
openai.api_base = API_BASE 
openai.api_version = API_VERSION
openai.api_key = AZURE_OPENAI_KEY

def generate_gpt_response(text, systemprompt):
    try:
        response = openai.ChatCompletion.create(
            engine=ENGINE,
            messages=[
                {"role": "system", "content": systemprompt},
                {"role": "user", "content": text}
            ],
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            top_p=TOP_P,
            frequency_penalty=FREQUENCY_PENALTY,
            presence_penalty=PRESENCE_PENALTY,
            stop=STOP
        )
        answer = response["choices"][0]["message"]["content"]
        # print(text, systemprompt, answer)
        if answer != '':
            return answer
        return text
    except Exception as e:
        print(text, systemprompt)
        print('GPT query failed: ', e)
        return DEFAULT_RESPONSE
    

In [6]:
story_prompt = 'Give me a short one paragraph of a list of mind-wobbling unknown facts about the given topic: '
topic = 'Mention Of Death In Christianity'
description = generate_gpt_response(topic, story_prompt)

In [7]:
description

'In Christianity, death is seen as a transition from earthly life to eternal life. Some mind-wobbling unknown facts about death in Christianity include the belief in the resurrection of the dead, the concept of purgatory, the idea of a final judgment, and the belief that death is not the end but a new beginning. Additionally, there are various interpretations of the afterlife, including the idea of heaven and hell, and the belief in the existence of angels and demons.'

In [8]:
import re

description = re.sub('[^A-Za-z0-9]+', ' ', description)
description

'In Christianity death is seen as a transition from earthly life to eternal life Some mind wobbling unknown facts about death in Christianity include the belief in the resurrection of the dead the concept of purgatory the idea of a final judgment and the belief that death is not the end but a new beginning Additionally there are various interpretations of the afterlife including the idea of heaven and hell and the belief in the existence of angels and demons '

In [9]:
description = re.sub('[0-9]+', '', description)
description

'In Christianity death is seen as a transition from earthly life to eternal life Some mind wobbling unknown facts about death in Christianity include the belief in the resurrection of the dead the concept of purgatory the idea of a final judgment and the belief that death is not the end but a new beginning Additionally there are various interpretations of the afterlife including the idea of heaven and hell and the belief in the existence of angels and demons '

In [10]:
!rm -rf ./test.wav

In [ ]:
from tqdm import tqdm
preset = "ultra_fast"
voice = 'tom'
voice_samples, conditioning_latents = load_voice(voice)
gen = tts.tts_with_preset(description, voice_samples=voice_samples, conditioning_latents=conditioning_latents, preset=preset)

torchaudio.save(f'test.wav', gen.squeeze(0).cpu(), 24000)


In [ ]:
from moviepy.editor import *

audio_file = './test.wav'
audioclip = AudioFileClip(audio_file)
video_duration = audioclip.duration

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
inpaint_model_list = [
    "stabilityai/stable-diffusion-2-inpainting",
    "runwayml/stable-diffusion-inpainting",
    "parlance/dreamlike-diffusion-1.0-inpainting",
    "ghunkins/stable-diffusion-liberty-inpainting",
    "ImNoOne/f222-inpainting-diffusers"
]
default_prompt = "A psychedelic jungle with trees that have glowing, fractal-like patterns, Simon stalenhag poster 1920s style, street level view, hyper futuristic, 8k resolution, hyper realistic"
default_negative_prompt = "frames, borderline, text, charachter, duplicate, error, out of frame, watermark, low quality, ugly, deformed, blur"

In [ ]:
model_id = inpaint_model_list[0]
pipe = StableDiffusionInpaintPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
)
pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(
    pipe.scheduler.config)
pipe = pipe.to("cuda")

pipe.safety_checker = None
pipe.enable_attention_slicing()
g_cuda = torch.Generator(device='cuda')

In [30]:
scene_list='Give me a list of 10 sentences comma separated describing mystical and fantasy visual scenes related to the provided description with uhd 64k quality written in them'
scenes = generate_gpt_response(description, scene_list)

In [31]:
scenes

'1. In the Level I multiverse, imagine infinite universes stretching out before you in a breathtaking display of cosmic wonder, each one unique and teeming with life.\n2. The Level II multiverse is a fantastical realm where the laws of physics are different in each universe, creating a kaleidoscope of colors and shapes that defy imagination.\n3. As you journey through the Level III multiverse, you encounter parallel universes with vastly different histories, where the course of events has diverged in ways that are both strange and wondrous.\n4. The multiverse theory suggests that our universe is just one of many, a tiny speck in a vast and infinite cosmos that stretches beyond our wildest dreams.\n5. In this multiverse, the laws of physics are not fixed but vary from universe to universe, creating a tapestry of worlds that is both beautiful and bewildering.\n6. As you explore the multiverse, you encounter strange and exotic creatures that defy description, each one a testament to the i

In [32]:
scenes = [i.split('.')[1].strip() for i in scenes.split('\n')]

In [41]:
scenes

['In the Level I multiverse, imagine infinite universes stretching out before you in a breathtaking display of cosmic wonder, each one unique and teeming with life',
 'The Level II multiverse is a fantastical realm where the laws of physics are different in each universe, creating a kaleidoscope of colors and shapes that defy imagination',
 'As you journey through the Level III multiverse, you encounter parallel universes with vastly different histories, where the course of events has diverged in ways that are both strange and wondrous',
 'The multiverse theory suggests that our universe is just one of many, a tiny speck in a vast and infinite cosmos that stretches beyond our wildest dreams',
 'In this multiverse, the laws of physics are not fixed but vary from universe to universe, creating a tapestry of worlds that is both beautiful and bewildering',
 'As you explore the multiverse, you encounter strange and exotic creatures that defy description, each one a testament to the infinite

In [45]:
filler = " that have glowing, fractal-like patterns, hyper futuristic, 8k resolution, hyper realistic"
scenes = [scene + filler for scene in scenes]

In [ ]:
!rm -rf test_dummy.mp4

In [48]:
zoom(scenes,
    default_negative_prompt,
    num_outpainting_steps = video_duration,
    guidance_scale = 7,
    num_inference_steps = 50,
    custom_init_image = None)

  0%|          | 0/50 [00:00<?, ?it/s]

Outpaint step: 1 / 63


  0%|          | 0/50 [00:00<?, ?it/s]

Outpaint step: 2 / 63


  0%|          | 0/50 [00:00<?, ?it/s]

...

Outpaint step: 62 / 63


  0%|          | 0/50 [00:00<?, ?it/s]

Outpaint step: 63 / 63


  0%|          | 0/50 [00:00<?, ?it/s]

'infinite_zoom_1709874173.6395056.mp4'

In [49]:
video_file = './test.mp4'
!rm -rf test_dummy.mp4

In [ ]:
def get_crop_height_and_width(x, y):
    while(int(x*16/9) > y):
        x = int(x * 0.95)
    y = int(x*16/9)
    return x, y

def resize_video(input_path, save_path, resize='crop'):
    v1 = VideoFileClip(input_path)
    aspect_ratio = v1.size[0]/v1.size[1]
    sp.call(f"rm -rf {save_path}", shell = True)
    if resize=='crop':
        crop_width, crop_height = get_crop_height_and_width(v1.size[0], v1.size[1])
        sp.call(f"ffmpeg -hide_banner -loglevel error -i {input_path} -vf 'crop={crop_width}:{crop_height},scale=720:1280' {save_path}", shell = True)
    else:
        if aspect_ratio < 0.5625:
            sp.call(f"ffmpeg -hide_banner -loglevel error -i {input_path} -vf 'split[original][copy];[copy]scale=((9*ih)/16):((9*ih*ih)/(16*iw)),crop=h=(16*iw/9),gblur=sigma=25[blurred];[blurred][original]overlay=(main_w-overlay_w)/2:(main_h-overlay_h)/2[final];[final]scale=720:1280' {save_path}", shell = True)
        elif aspect_ratio > 0.5625:
            sp.call(f"ffmpeg -hide_banner -loglevel error -i {input_path} -vf 'split[original][copy];[copy]scale=(16/9)*iw*iw/ih:(16*iw/9),crop=w=(9*ih/16),gblur=sigma=25[blurred];[blurred][original]overlay=(main_w-overlay_w)/2:(main_h-overlay_h)/2[final];[final]scale=720:1280' {save_path}", shell = True)
        else:
            sp.call(f"ffmpeg -hide_banner -loglevel error -i {input_path} -vf '[original]scale=720:1280' {save_path}", shell = True)

In [ ]:
resize_video('test_dummy.mp4', video_file)

In [ ]:
!rm -rf story.mp4

In [51]:
videoclip = VideoFileClip(video_file).subclip(0, video_duration) 

# audioclip = audioclip.audio_loop(duration = videoclip.duration)

new_audioclip = CompositeAudioClip([audioclip])
videoclip.audio = new_audioclip
videoclip.write_videofile("story.mp4",
                          codec='libx264', 
                          audio_codec='aac',
                          remove_temp=True,
                          verbose=False,
                          logger=None)